# FP-Growth Algorithm:-

Frequent Pattern Growth (FP-Growth) is an algorithm for mining frequent itemsets in a transaction database. It is an efficient alternative to the Apriori algorithm, which relies on candidate generation and multiple database scans. FP-Growth addresses the inefficiencies of Apriori by constructing a compact data structure called the **FP-tree (Frequent Pattern Tree)** and directly mining the frequent patterns from it.

---
## Why Was FP-Growth Introduced?
The FP-Growth algorithm was introduced to overcome the following challenges of Apriori:
1. **High Computational Cost**: Apriori generates a large number of candidate itemsets, even for small datasets, leading to increased computation time.
2. **Multiple Database Scans**: Apriori requires multiple passes over the database to compute the support of itemsets, which is resource-intensive.
3. **Memory Usage**: Storing and processing candidate itemsets becomes infeasible as the number of items in the dataset increases.

FP-Growth solves these issues by:
- Reducing the number of database scans to two.
- Avoiding candidate generation entirely.
- Using a compact representation of the dataset (FP-tree) to efficiently mine frequent itemsets.

---

## Key Concepts
### Frequent Itemsets
- Frequent itemsets are sets of items that occur together in transactions with a frequency greater than or equal to a predefined threshold, called **minimum support**.

  
### Support
- Support is the proportion of transactions in the database that contain a particular itemset. It is defined as:

$$
\text{Support}(A) = \frac{\text{Count of transactions containing } A}{\text{Total number of transactions}}
$$


### FP-Tree
- The FP-tree is a compressed representation of the transaction database.
- It stores only the frequent items in a hierarchical structure, reducing redundancy and allowing frequent patterns to be mined directly.

---

## How FP-Growth Works
The FP-Growth algorithm works in two main steps:
1. **Constructing the FP-Tree**:
   - Scan the database to compute the frequency of all items.
   - Remove items that do not meet the minimum support threshold.
   - Sort the remaining items by descending frequency and construct the FP-tree.

2. **Mining the FP-Tree**:
   - Traverse the FP-tree to extract frequent itemsets using a bottom-up approach.
   - Conditional FP-trees are built for each frequent item, enabling the discovery of all frequent patterns.

## Steps in Detail
### Step 1: FP-Tree Construction
1. **First Pass (Item Frequency Count)**:
   - Count the frequency of each item in the dataset.
   - Filter out items with support less than the minimum support threshold.

2. **Second Pass (Tree Construction)**:
   - Sort items in each transaction by descending order of frequency.
   - Insert each transaction into the FP-tree. If a prefix of the transaction already exists, increment the count of the existing nodes; otherwise, create new nodes.

### Step 2: Mining Frequent Patterns
1. **Conditional Pattern Base**:
   - For each frequent item, extract its **conditional pattern base** (a subset of the database that contributes to its frequency).

2. **Conditional FP-Tree**:
   - Build a smaller FP-tree from the conditional pattern base.

3. **Recursive Mining**:
   - Recursively mine the conditional FP-tree to find frequent patterns.

4. **Combine Patterns**:
   - Combine the mined patterns to form the complete set of frequent itemsets.

---

## Advantages of FP-Growth
- **Efficient**: FP-Growth avoids the costly candidate generation process and reduces the number of database scans.
- **Scalable**: Works well with large datasets due to its compact representation of data.
- **Memory-Friendly**: Uses the FP-tree to store only frequent items, reducing memory requirements.

## Disadvantages of FP-Growth
- **Complex Tree Construction**: Building the FP-tree can be complex and time-consuming for very large datasets.
- **Recursive Nature**: The recursive mining process may lead to high memory usage in certain cases.


## Applications of FP-Growth
- **Market Basket Analysis**: Understanding customer purchasing patterns.
- **Recommendation Systems**: Recommending items based on frequent co-occurrence.
- **Fraud Detection**: Identifying unusual patterns in financial transactions.


---

FP-Growth is a powerful and efficient algorithm for frequent itemset mining, particularly suitable for large datasets. By leveraging the compact FP-tree structure, it eliminates the need for candidate generation and reduces the computational cost significantly.



In [1]:

# importing libraries

from collections import defaultdict


## Step 1: Define FP-tree Node
The FP-tree node represents each item in a transaction, storing:
- `item`: The item name.
- `count`: Count of the item.
- `parent`: Pointer to the parent node.
- `children`: Dictionary of child nodes.
- `link`: Pointer for the linked list in the header table.

In [2]:
# define the fptree node class 

class TreeNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def increment(self, count):
        self.count += count


## Step 2: Build the FP-tree
1. Count item frequencies.
2. Filter items below the minimum support.
3. Sort transactions by item frequency.
4. Build the tree and update the header table.


In [3]:
def build_tree(transactions, min_support):
    item_counts = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    # Filter items below min_support
    item_counts = {item: count for item, count in item_counts.items() if count >= min_support}
    if not item_counts:
        return None, None

    # Sort transactions by item frequency
    ordered_transactions = []
    for transaction in transactions:
        ordered = sorted([item for item in transaction if item in item_counts], key=lambda x: -item_counts[x])
        if ordered:
            ordered_transactions.append(ordered)

    root = TreeNode(None, 0, None)
    header_table = {item: None for item in item_counts}

    for transaction in ordered_transactions:
        add_to_tree(transaction, root, header_table)

    return root, header_table


def add_to_tree(transaction, root, header_table):
    first_item = transaction[0]
    if first_item not in root.children:
        root.children[first_item] = TreeNode(first_item, 1, root)
        update_links(header_table, root.children[first_item])
    else:
        root.children[first_item].increment(1)

    if len(transaction) > 1:
        add_to_tree(transaction[1:], root.children[first_item], header_table)


def update_links(header_table, node):
    if header_table[node.item] is None:
        header_table[node.item] = node
    else:
        current = header_table[node.item]
        while current.link is not None:
            current = current.link
        current.link = node


## Step 3: Mine the FP-tree
1. For each item in the header table:
   - Build its conditional pattern base.
   - Create a conditional FP-tree.
   - Recursively mine frequent patterns.
2. Combine frequent patterns from conditional trees.


In [5]:
def mine_tree(header_table, min_support):
    patterns = {}
    for item, node in header_table.items():
        pattern_base = []
        while node is not None:
            path = []
            parent = node.parent
            while parent is not None and parent.item is not None:
                path.append(parent.item)
                parent = parent.parent
            if path:
                pattern_base.append((path, node.count))
            node = node.link

        conditional_tree, conditional_header = build_conditional_tree(pattern_base, min_support)

        if conditional_tree:
            sub_patterns = mine_tree(conditional_header, min_support)
            for pattern, count in sub_patterns.items():
                patterns[tuple(sorted([item] + list(pattern)))] = count

        patterns[(item,)] = patterns.get((item,), 0) + sum(count for _, count in pattern_base)

    return patterns


def build_conditional_tree(pattern_base, min_support):
    conditional_items = defaultdict(int)
    for path, count in pattern_base:
        for item in path:
            conditional_items[item] += count

    conditional_items = {item: count for item, count in conditional_items.items() if count >= min_support}
    if not conditional_items:
        return None, None

    conditional_transactions = []
    for path, count in pattern_base:
        ordered = [item for item in path if item in conditional_items]
        conditional_transactions.extend([ordered] * count)

    return build_tree(conditional_transactions, min_support)


## Step 4: FP-Growth Algorithm
Integrates the above steps to:
1. Build the FP-tree.
2. Mine frequent patterns from the FP-tree.


In [6]:
def fp_growth(transactions, min_support):
    tree, header_table = build_tree(transactions, min_support)
    if not tree:
        return {}

    return mine_tree(header_table, min_support)


## Example Usage
The following example demonstrates FP-Growth with a set of transactions and a minimum support threshold.


In [7]:
transactions = [
    ['Milk', 'Bread', 'Butter'],
    ['Bread', 'Butter', 'Cheese'],
    ['Milk', 'Bread', 'Butter', 'Cheese'],
    ['Milk', 'Bread'],
    ['Bread', 'Butter', 'Cheese', 'Eggs'],
    ['Milk', 'Bread', 'Cheese'],
    ['Milk', 'Butter', 'Cheese'],
    ['Bread', 'Butter'],
]

min_support = 2

frequent_itemsets = fp_growth(transactions, min_support)

# Display Results
print("Frequent Itemsets:")
for itemset, support in frequent_itemsets.items():
    print(f"{set(itemset)}: {support}")


Frequent Itemsets:
{'Butter', 'Bread', 'Milk'}: 0
{'Butter', 'Milk'}: 2
{'Milk', 'Bread'}: 0
{'Milk'}: 5
{'Bread'}: 0
{'Butter', 'Bread'}: 0
{'Butter'}: 5
{'Butter', 'Cheese'}: 0
{'Butter', 'Bread', 'Cheese'}: 0
{'Cheese', 'Bread'}: 3
{'Cheese', 'Bread', 'Milk'}: 0
{'Milk', 'Butter', 'Cheese'}: 1
{'Cheese', 'Milk'}: 3
{'Cheese'}: 5
